In [76]:
import pandas as pd
import numpy as np

import import_ipynb
import preprocessamentofuncoes as pp

### Definindo as Funções Auxiliares

In [11]:
# Juntando bases (merge)
def juntarBases(base_1, base_2):
    covidBalanced = pd.merge(base_1, base_2, how = 'outer')
    covidBalanced.reset_index(inplace=True, drop = True)
    return covidBalanced

In [12]:
# Excluindo os dias vazios
def removerAtributosFaltosos(covidData, atributo):
    instanciasVazias = covidData.loc[covidData[atributo].isnull()]
    covidData = covidData.drop(instanciasVazias.index)
    return covidData

In [13]:
def converterDiasSintomas(covidData, diasSintomas, novaColuna):
    covidData.loc[((covidData[diasSintomas] >= 0) & (covidData[diasSintomas] <= 3)), novaColuna] = 0
    covidData.loc[((covidData[diasSintomas] >= 4) & (covidData[diasSintomas] <= 6)), novaColuna] = 1
    covidData.loc[((covidData[diasSintomas] >= 7) & (covidData[diasSintomas] <= 9)), novaColuna] = 2
    covidData.loc[((covidData[diasSintomas] >= 10) & (covidData[diasSintomas] <= 12)), novaColuna] = 3
    covidData.loc[((covidData[diasSintomas] >= 13) & (covidData[diasSintomas] <= 15)), novaColuna] = 4
    covidData.loc[((covidData[diasSintomas] >= 16) & (covidData[diasSintomas] <= 18)), novaColuna] = 5
    covidData.loc[((covidData[diasSintomas] > 18)), novaColuna] = 6
    return covidData

In [14]:
def salvarDataframeEmPlanilha(covidDataBalanced, url):
    write = pd.ExcelWriter(url)
    covidDataBalanced.to_excel(write, 'dados', index = False)
    write.save()

In [15]:
def balanceandoDados(urlBase, colunasRemovidas, uf):
    print("Iniciando - Base ", uf)
    covidData = pd.read_excel(urlBase)
    
    # Removendo as colunas destacadas
    #covidData = covidData.drop(columns = colunasRemovidas)
    
    # Excluindo os dias vazios
    covidData = removerAtributosFaltosos(covidData, 'diasSintomas')
    
    # Removendo os dias outliers
    diasSintomasOutliers = covidData.loc[((covidData['diasSintomas'] > 30))]
    covidData = covidData.drop(diasSintomasOutliers.index)
    
    # Converter os dias de sintomas em faixas
    covidData = converterDiasSintomas(covidData, 'diasSintomas', 'faixaDiasSintomas')
    
    # Removendo o atributo Dias de Sintomas 
    covidData = covidData.drop(columns = 'diasSintomas')
    
    print("Base Total: \n", covidData['evolucaoCaso'].value_counts(), "\n====================")
    
    # Balanceamento - OC 60/40
    covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 0, 40, 1)
    covidDataBalanced = removerAtributosFaltosos(covidDataBalanced, 'faixaDiasSintomas')
    salvarDataframeEmPlanilha(covidDataBalanced, 'PreProcessamento/balanceadas/OC/ba-oc-6040-' + uf + '.xlsx')
    print('OC 6040 - ', covidDataBalanced.shape)
    print(covidDataBalanced['evolucaoCaso'].value_counts())
    
    # Balanceamento - OC 70/30
    covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 0, 30, 1)
    covidDataBalanced = removerAtributosFaltosos(covidDataBalanced, 'faixaDiasSintomas')
    salvarDataframeEmPlanilha(covidDataBalanced, 'PreProcessamento/balanceadas/OC/ba-oc-7030-' + uf + '.xlsx')
    print('OC 7030 - ', covidDataBalanced.shape)
    print(covidDataBalanced['evolucaoCaso'].value_counts())

    # Balanceamento - IC 60/40
    covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 2, 40, 1)
    covidDataBalanced = removerAtributosFaltosos(covidDataBalanced, 'faixaDiasSintomas')
    salvarDataframeEmPlanilha(covidDataBalanced, 'PreProcessamento/balanceadas/IC/ba-ic-6040-' + uf + '.xlsx')
    print('IC 6040 - ', covidDataBalanced.shape)
    print(covidDataBalanced['evolucaoCaso'].value_counts())

    # Balanceamento - IC 70/30
    covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 2, 30, 1)
    covidDataBalanced = removerAtributosFaltosos(covidDataBalanced, 'faixaDiasSintomas')
    salvarDataframeEmPlanilha(covidDataBalanced, 'PreProcessamento/balanceadas/IC/ba-ic-7030-' + uf + '.xlsx')
    print('IC 7030 - ', covidDataBalanced.shape)
    print(covidDataBalanced['evolucaoCaso'].value_counts())
    print("=====================================")
  
    

### Definindo as Colunas Removidas e as Bases Utilizadas

In [9]:
colunasRemovidas = ['source_id', 'dataInicioSintomas', 'dataNotificacao', 'idade',
                    'sintomas', 'outrosSintomas', 'condicoes', 'outrasCondicoes',
                    'dataPrimeiraDose', 'dataSegundaDose', 'classificacaoFinal', 
                    'outrosSintomas', 'outrasCondicoes']

In [7]:
bases = ['sus-ac-pp.xlsx', 'sus-al-pp.xlsx', 'sus-am-pp.xlsx', 'sus-ap-pp.xlsx', 
         'sus-ba-pp.xlsx', 'sus-ce-pp.xlsx', 'sus-df-pp.xlsx', 
         'sus-go-pp.xlsx', 'sus-ma-pp.xlsx', 'sus-mg-pp.xlsx', 'sus-ms-pp.xlsx', 
         'sus-mt-pp.xlsx', 'sus-pa-pp.xlsx', 'sus-pb-pp.xlsx', 'sus-pe-pp.xlsx', 
         'sus-pi-pp.xlsx', 'sus-rj-pp.xlsx', 'sus-rn-pp.xlsx', 
         'sus-ro-pp.xlsx', 'sus-rr-pp.xlsx', 'sus-rs-pp.xlsx', 'sus-sc-pp.xlsx', 
         'sus-se-pp.xlsx', 'sus-to-pp.xlsx']


ufs = ['ac', 'al', 'am', 'ap', 'ba', 'ce', 'df',  
      'go', 'ma', 'mg', 'ms', 'mt', 'pa', 'pb', 'pe',
      'pi', 'rj', 'rn', 'ro', 'rr', 'rs', 'sc', 
       'se', 'to']

### Percorrendo as bases e realizando o balanceamento

In [11]:
url_padrao = 'PreProcessamento/'
for i in range(0, len(bases)):
    url = url_padrao + bases[i]
    balanceandoDados(url, colunasRemovidas, ufs[i])

Iniciando - Base  ac
Base Total: 
 1    20819
0       86
2       17
Name: evolucaoCaso, dtype: int64 
OC 6040 -  (215, 49)
1    129
0     86
Name: evolucaoCaso, dtype: int64
OC 7030 -  (286, 49)
1    200
0     86
Name: evolucaoCaso, dtype: int64
IC 6040 -  (42, 49)
1    25
2    17
Name: evolucaoCaso, dtype: int64
IC 7030 -  (56, 49)
1    39
2    17
Name: evolucaoCaso, dtype: int64
Iniciando - Base  al
Base Total: 
 1    24532
0      139
2       26
Name: evolucaoCaso, dtype: int64 
OC 6040 -  (347, 49)
1    208
0    139
Name: evolucaoCaso, dtype: int64
OC 7030 -  (463, 49)
1    324
0    139
Name: evolucaoCaso, dtype: int64
IC 6040 -  (65, 49)
1    39
2    26
Name: evolucaoCaso, dtype: int64
IC 7030 -  (86, 49)
1    60
2    26
Name: evolucaoCaso, dtype: int64
Iniciando - Base  am
Base Total: 
 1    63468
0       71
2       47
Name: evolucaoCaso, dtype: int64 
OC 6040 -  (177, 49)
1    106
0     71
Name: evolucaoCaso, dtype: int64
OC 7030 -  (236, 49)
1    165
0     71
Name: evolucaoCaso,

OC 7030 -  (1756, 49)
1    1229
0     527
Name: evolucaoCaso, dtype: int64
IC 6040 -  (85, 49)
1    51
2    34
Name: evolucaoCaso, dtype: int64
IC 7030 -  (113, 49)
1    79
2    34
Name: evolucaoCaso, dtype: int64
Iniciando - Base  rr
Base Total: 
 1    7037
0     132
2      13
Name: evolucaoCaso, dtype: int64 
OC 6040 -  (330, 49)
1    198
0    132
Name: evolucaoCaso, dtype: int64
OC 7030 -  (440, 49)
1    308
0    132
Name: evolucaoCaso, dtype: int64
IC 6040 -  (32, 49)
1    19
2    13
Name: evolucaoCaso, dtype: int64
IC 7030 -  (43, 49)
1    30
2    13
Name: evolucaoCaso, dtype: int64
Iniciando - Base  rs
Base Total: 
 1    339983
0       219
2       189
Name: evolucaoCaso, dtype: int64 
OC 6040 -  (547, 49)
1    328
0    219
Name: evolucaoCaso, dtype: int64
OC 7030 -  (729, 49)
1    510
0    219
Name: evolucaoCaso, dtype: int64
IC 6040 -  (472, 49)
1    283
2    189
Name: evolucaoCaso, dtype: int64
IC 7030 -  (630, 49)
1    441
2    189
Name: evolucaoCaso, dtype: int64
Iniciando - 

In [ ]:
# balanceandoDados(urlBase, colunasRemovidas, uf)


### Salvando as proporções das bases

In [16]:
dfMetricas = pd.DataFrame()
for i in range(0, len(bases)):
    print('base ', ufs[i])
    row = {}
    url = urlPadrao + bases[i]
    covidData = pd.read_excel(url)
    evolucao = covidData['evolucaoCaso'].value_counts()
    print(evolucao)

    row['UF'] = ufs[i].upper()
    row['Instâncias'] = covidData.shape[0]
    row['Curado'] = evolucao[1]
    row['Interado'] = evolucao[2]
    row['Óbito'] = evolucao[0]

    dfMetricas = pd.concat([dfMetricas, pd.DataFrame([row])])
    print("=================================")

base  ac
1    23760
0       92
2       18
Name: evolucaoCaso, dtype: int64
base  al
1    27138
0      161
2       28
Name: evolucaoCaso, dtype: int64
base  am
1    71535
0      138
2       62
Name: evolucaoCaso, dtype: int64
base  ap
1    12622
0      186
2       33
Name: evolucaoCaso, dtype: int64
base  ba
1    136890
0      1133
2       426
Name: evolucaoCaso, dtype: int64
base  ce


KeyboardInterrupt: 

In [ ]:
salvarDataframeEmPlanilha(dfMetricas, 'metricas.xlsx')

### Avaliando as bases de SP

In [50]:
url = 'PreProcessamento/balanceadas/SP/'
file = 'new-pp-ic-6040-'

dataset_1 = pd.read_excel(url + file + 'sp.xlsx')
dataset_2 = pd.read_excel(url + file + 'sp2.xlsx')
merged_dataset = juntarBases(dataset_1, dataset_2)

In [54]:
merged_dataset['evolucaoCaso'].value_counts()
#merged_dataset.shape

1    1793
2     727
Name: evolucaoCaso, dtype: int64

In [55]:
uf = 'sp'

# Excluindo os dias vazios
covidData = merged_dataset
covidData = removerAtributosFaltosos(covidData, 'diasSintomas')
    
# Removendo os dias outliers
diasSintomasOutliers = covidData.loc[((covidData['diasSintomas'] > 30))]
covidData = covidData.drop(diasSintomasOutliers.index)

# Converter os dias de sintomas em faixas
covidData = converterDiasSintomas(covidData, 'diasSintomas', 'faixaDiasSintomas')

# Removendo o atributo Dias de Sintomas 
covidData = covidData.drop(columns = 'diasSintomas')

# Salvando a base balanceada
covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 2, 40, 1)
covidDataBalanced = removerAtributosFaltosos(covidDataBalanced, 'faixaDiasSintomas')
salvarDataframeEmPlanilha(covidDataBalanced, 'PreProcessamento/balanceadas/IC/ba-ic-6040-' + uf + '.xlsx')

In [56]:
covidDataBalanced['evolucaoCaso'].value_counts()

1    1090
2     727
Name: evolucaoCaso, dtype: int64

In [40]:
covidDataBalanced.shape

(3577, 38)

### Gerando as bases com Smote

In [77]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

def aplicarSmote(base, alvo):

    X_smt = base.drop(alvo, axis = 1)
    y_smt = base[alvo]
    
    # Contar o número de ocorrências
    # np.bincount(y_smt)
    
    # Carregar o Smote
    smt = SMOTE()
    
    # Aplicar o Smote
    X_smt, y_smt = smt.fit_resample(X_smt, y_smt)
    
    # Contar o número de ocorrências
    # np.bincount(y_smt)
    
    # Convertendo em DF
    smoteBase = pd.DataFrame(X_smt)
    smoteBase[alvo] = y_smt
    
    return smoteBase

In [93]:
file_url = 'PreProcessamento/balanceadas/IC/ba-ic-7030-sp.xlsx'
file_name = 'sm-ic-7030-sp.xlsx'
alvo = 'evolucaoCaso'

dataset = pd.read_excel(file_url)
dataset[alvo].value_counts()

1    1696
2     727
Name: evolucaoCaso, dtype: int64

In [94]:
dataset_smote = aplicarSmote(dataset, alvo)
dataset_smote[alvo].value_counts()

2    1696
1    1696
Name: evolucaoCaso, dtype: int64

In [95]:
salvarDataframeEmPlanilha(dataset_smote, 'PreProcessamento/balanceadas/Smote/' + file_name)